# Capstone Project from E. Wesner

This notebook will be mainly used for the capstone project of Coursera Data Science

In [1]:
import pandas as pd
import numpy as np
print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


# Neighborhoods of Toronto "Dataimport and Cleaning"

**Import the Data from Wikipedia**

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_data = requests.get (URL)
soup = BeautifulSoup(html_data.content, 'html.parser')


In [4]:
#Empty dataframe with cols
neighborhoods_toronto = pd.DataFrame(columns = ["PostalCode","Borough","Neighborhood"])

#from the Soup pick the relevant data
table = soup.find_all("tbody")[0]

#extract the table and values
for row in table.find_all("tr"):
    col = row.find_all("td")
    textlist = [ele.text.strip()for ele in col]
    if textlist:                    #falls in text was drin steht, also nicht [] leer ist
        #print(text)
        for i in range (len(textlist)):
            einzeln = textlist[i]
            postcode = einzeln[0:3]
            schneiden = einzeln[3:].find("(")
            boro = einzeln[3:schneiden+3]   
            neigh = einzeln[schneiden+4:-1]
            neigh = neigh.replace(' / ', ' , ')
            neighborhoods_toronto = neighborhoods_toronto.append({"PostalCode":postcode,"Borough":boro,"Neighborhood":neigh}, 
                                                        ignore_index = True)
neighborhoods_toronto.shape

(180, 3)

In [5]:
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,,Not assigne
1,M2A,,Not assigne
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"


In [6]:
#Not assignet delete
neighborhoods_toronto1=neighborhoods_toronto.replace('Not assigne', np.NaN)
neighborhoods_toronto2=neighborhoods_toronto1.replace('', np.NaN)
neighborhoods_toronto=neighborhoods_toronto2.dropna(subset=["Borough"],axis=0)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [7]:
neighborhoods_toronto.shape

(103, 3)

In [8]:
#create the new index
neighborhoods_toronto.reset_index(drop=True,inplace=True)
neighborhoods_toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [9]:
#searching for not assigned neighborhood but a borough
missing_data=neighborhoods_toronto.isnull()
for columns in missing_data.columns.values.tolist():
    print (columns)
    print (missing_data[columns].value_counts())
    print(" ")

PostalCode
False    103
Name: PostalCode, dtype: int64
 
Borough
False    103
Name: Borough, dtype: int64
 
Neighborhood
False    103
Name: Neighborhood, dtype: int64
 


No such data was found

In [10]:
print("The Shape of the DataFrame is: " + str(neighborhoods_toronto.shape))

The Shape of the DataFrame is: (103, 3)


# Part 2   Get the Latidude and Longitude

In [18]:
#Empty dataframe with cols
neighborhoods_coordinates = pd.DataFrame(columns = ["Postal Code","Latitude","Longitude"])
neighborhoods_coordinates.head()

,Postal Code,Latitude,Longitude


**Tried to use geocoder, but I didn't succeed**

In [ ]:
!pip install geocoder
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

for i in range (neighborhoods_toronto.shape[0]):
    postal_code = neighborhoods_toronto.at[neighborhoods_toronto.index[i],'Postal Code'] 
    print(postal_code)
    #loop until you get the coordinates
    while(lat_lng_coords is None):        
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    neighborhoods_coordinates = neighborhoods_coordinates.append({"Postal Code":postal_code,"Latitude":latitude,"Longitude":longitude}, 
                                                        ignore_index = True)
neighborhoods_coordinates.shape

**This code was created from the Notebook by importing the CSV-file Geospatial_Coordinates.csv**

In [22]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_0041bdc6e7bf463992b36c4279e007e7 = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_0041bdc6e7bf463992b36c4279e007e7 = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_0041bdc6e7bf463992b36c4279e007e7 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='_WvAzxSMMbyyvT7Bo77QzhnGJ2DPckT5a9RkQmE9rCnN',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_0041bdc6e7bf463992b36c4279e007e7)

body = client_0041bdc6e7bf463992b36c4279e007e7.get_object(Bucket='courseracapstonedatascience-donotdelete-pr-mgnh2uarlhwkrt',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
#Merge the Coordinates with the Neighborhoods: 

neighborhoods_merged = neighborhoods_toronto
neighborhoods_merged = neighborhoods_merged.join(df_data_1.set_index('Postal Code'), on = 'PostalCode')
neighborhoods_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
